# A more complete demo of Speasy

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import speasy as spz
from speasy.products import SpeasyVariable
from typing import List
import numpy as np
from datetime import datetime

#plt.rcParams["figure.figsize"] = (20, 4)

## Few products from CDAWeb

You can browse CDAWeb inventory using your python environement completion and get any 'variable'(CDA name) or 'parameter' (Speasy name) on any valid time interval.
Then most SpeasyVariables can be plotted directly (using matplotlib backend).

In [ ]:
solo_fgm: SpeasyVariable = spz.get_data(
    spz.inventories.tree.cda.Solar_Orbiter.SOLO.MAG.SOLO_L2_MAG_RTN_NORMAL.B_RTN,
    "2022-03-01",
    "2022-03-01T12",
)
plt.figure()
solo_fgm.plot()
plt.tight_layout()
plt.show()


It is quite common to have **fill values**, you can replace them by **NaN** by simply calling `replace_fillval_by_nan`

In [ ]:
swa_pas_density: SpeasyVariable = spz.get_data(
    spz.inventories.tree.cda.Solar_Orbiter.SOLO.SWA_PAS_MOM.SOLO_LL02_SWA_PAS_MOM.SWA_PAS_DENSITY,
    "2022-03-01",
    "2022-03-01T12",
)
plt.figure("With 1e31 fill values")
swa_pas_density.plot()
plt.tight_layout()
plt.show()
plt.figure()
swa_pas_density.replace_fillval_by_nan().plot()
plt.tight_layout()
plt.show()


You can also get and plot spectograms

In [ ]:
mms2_des_energyspectr_omni_fast: SpeasyVariable = spz.get_data(
    spz.inventories.tree.cda.MMS.MMS2.DES.MMS2_FPI_FAST_L2_DES_MOMS.mms2_des_energyspectr_omni_fast,
    "2022-03-02",
    "2022-03-03",
)
plt.figure()
mms2_des_energyspectr_omni_fast.plot(cmap="viridis")
plt.tight_layout()
plt.show()


## SSCWeb Trajectory example

In [ ]:
plt.figure()
spz.get_data(
    spz.inventories.tree.ssc.Trajectories.cluster4, "2010-11-01", "2010-11-30"
).plot()
plt.tight_layout()
plt.show()

## CSA spectrogram example

Speasy also support the Cluster Science Archive which mean you can get most Cluster 2 and Double Star products

In [ ]:
plt.figure()
spz.get_data(
    spz.inventories.tree.csa.Cluster.Cluster_1.CIS_HIA1.C1_CP_CIS_HIA_HS_1D_PEF.flux__C1_CP_CIS_HIA_HS_1D_PEF,
    "2006-11-01",
    "2006-11-02",
).plot(cmap="jet")
plt.tight_layout()
plt.show()

## Speasy can download several products at once for a given interval

Speasy `get_data` function is quite flexible and can download several products at once

In [ ]:
products = [
    spz.inventories.tree.cda.MMS.MMS1.FGM.MMS1_FGM_SRVY_L2.mms1_fgm_b_gse_srvy_l2_clean,
    spz.inventories.tree.cda.MMS.MMS2.FGM.MMS2_FGM_SRVY_L2.mms2_fgm_b_gse_srvy_l2_clean,
    spz.inventories.tree.cda.MMS.MMS3.FGM.MMS3_FGM_SRVY_L2.mms3_fgm_b_gse_srvy_l2_clean,
    spz.inventories.tree.cda.MMS.MMS4.FGM.MMS4_FGM_SRVY_L2.mms4_fgm_b_gse_srvy_l2_clean,
]

#fig = plt.figure(figsize=(20, 8))
fig = plt.figure()
gs = fig.add_gridspec(4, hspace=0)
axes = gs.subplots(sharex=True, sharey=True)
mms_fgm_b_gse_srvy: List[SpeasyVariable] = spz.get_data(
    products, "2019-01-02T14", "2019-01-02T20")
for var, ax in zip(mms_fgm_b_gse_srvy, axes):
    var["Bx GSE", "By GSE", "Bz GSE"].replace_fillval_by_nan().plot(ax=ax)
plt.tight_layout()
plt.show()


## Several product for several dates is also supported and get_data preserves input product list shape

Speasy `get_data` function support any nesting depth of product and time ranges lists, it will always explore products first then for each product get data for each given time range

In [ ]:
data_preserve_shape: List[List[List[SpeasyVariable]]] = spz.get_data(
    [
        [
            spz.inventories.tree.cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.Beta,
            spz.inventories.tree.cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.T,
        ],
        [
            spz.inventories.tree.cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.E,
            spz.inventories.tree.cda.OMNI_Combined_1AU_IP_Data__Magnetic_and_Solar_Indices.OMNI_1AU_IP_Data.IMF_and_Plasma_data.OMNI_HRO_1MIN.Pressure,
        ],
    ],
    [["2010-01-02", "2010-01-02T10"], ["2009-08-02", "2009-08-02T10"]]
)
data_preserve_shape


In [ ]:
#fig = plt.figure(figsize=(20, 6))
fig = plt.figure()
gs = fig.add_gridspec(1, 2)
left_gs = gs[0, 0].subgridspec(4, 1, hspace=0)
right_gs = gs[0, 1].subgridspec(4, 1, hspace=0)
left_ax = left_gs.subplots(sharex=True, sharey=False)
right_ax = right_gs.subplots(sharex=True, sharey=False)

data_preserve_shape[0][0][0].replace_fillval_by_nan().plot(ax=left_ax[0])
data_preserve_shape[0][1][0].replace_fillval_by_nan().plot(ax=left_ax[1])
data_preserve_shape[1][0][0].replace_fillval_by_nan().plot(ax=left_ax[2])
data_preserve_shape[1][1][0].replace_fillval_by_nan().plot(ax=left_ax[3])

data_preserve_shape[0][0][1].replace_fillval_by_nan().plot(ax=right_ax[0])
data_preserve_shape[0][1][1].replace_fillval_by_nan().plot(ax=right_ax[1])
data_preserve_shape[1][0][1].replace_fillval_by_nan().plot(ax=right_ax[2])
data_preserve_shape[1][1][1].replace_fillval_by_nan().plot(ax=right_ax[3])
plt.tight_layout()
plt.show()

## Some SpeasyVariable tricks


### Attributes

A SpeasyVariable is close to a pandas DataFrame, it has few attributes, most of them are extracted from underlying CDF or CSV files generated by corresponding web-services:

In [ ]:
solo_fgm: SpeasyVariable = spz.get_data(
    spz.inventories.tree.cda.Solar_Orbiter.SOLO.MAG.SOLO_L2_MAG_RTN_NORMAL.B_RTN,
    "2022-03-01",
    "2022-03-01T12",
)
print("===========================================")
print(f"Name:         {solo_fgm.name}")
print(f"Columns:      {solo_fgm.columns}")
print(f"Values Unit:  {solo_fgm.unit}")
print(f"Memory usage: {solo_fgm.nbytes} Bytes")
print(f"Axes Labels:  {solo_fgm.axes_labels}")
print("-------------------------------------------")
print(f"Meta-data:    {solo_fgm.meta}")
print("-------------------------------------------")
print(f"Time Axis:    {solo_fgm.time[:3]}")
print("-------------------------------------------")
print(f"Values:       {solo_fgm.values[:3]}")
print("===========================================")

### Methods

SpeasyVaraible class is not meant to be as fully-featured as pandas DataFrame or Xarray DataArray but still has few useful features

- you can make a (deep)copy of a SpeasyVariable:

In [ ]:
solo_fgm2 = solo_fgm.copy()
solo_fgm2 is solo_fgm, solo_fgm == solo_fgm2

- you can build a new variable with only a subset of columns of a given variable

In [ ]:
solo_fgm2 = solo_fgm.filter_columns(["B_r", "B_t"])
print(solo_fgm.columns, solo_fgm2.columns)

In [ ]:
plt.figure("Variable with filtered columns")
solo_fgm.filter_columns(["B_r", "B_t"]).plot()
plt.tight_layout()
plt.show()

- you can export simple time-series to Astropy Tables or pandas DataFrames

In [ ]:
solo_fgm.to_astropy_table()[:10].show_in_notebook()

In [ ]:
solo_fgm.to_dataframe().head()

- you can slice a SpeasyVariable by index or time

In [ ]:
solo_fgm[:5].to_dataframe()

In [ ]:
solo_fgm[np.datetime64("2022-03-01T00:00:00"): np.datetime64("2022-03-01T00:00:00.6")].to_dataframe()

In [ ]:
solo_fgm[datetime(2022,3,1): datetime(2022,3,1,microsecond=600000)].to_dataframe()

- you can convert values into Astropy Quantity:

In [ ]:
solo_fgm[:5].unit_applied().values
